In [1]:
from langfuse import Langfuse
from fsm_core.llm_common import get_sync_client
from compiler.core import Compiler
from application import Application

In [2]:
langfuse_client = Langfuse()
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
m_client = get_sync_client("bedrock", client_params={"aws_profile":"dev", "aws_region": "us-west-2"})

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [ ]:
from agent.application import InteractionMode
builder = Application(m_client, compiler, interaction_mode=InteractionMode.INTERACTIVE)

In [4]:
prepare_res = builder.prepare_bot(["bot that greets me by name"], bot_id="1337_420", capabilities=[])

In [5]:
prepare_res

ApplicationPrepareOut(refined_description=RefineOut(refined_description='', error_output=None), capabilities=CapabilitiesOut(capabilities=[], error_output=None), typespec=TypespecOut(reasoning="The user requested a bot that greets them by name. This is a simple use case where the bot needs to:\n1. Store and retrieve the user's name\n2. Generate appropriate greetings based on the user's name\n3. Potentially support different greeting styles or times of day\n\nFor this bot, we need:\n- A model to represent the user's information (name)\n- A model for greeting options (time of day, style)\n- Functions to set the user's name and to generate greetings\n\nThe interface should support:\n- Setting the user's name\n- Getting a greeting with the user's name\n- Possibly updating greeting preferences", typespec_definitions='model UserInfo {\n  name: string;\n}\n\nmodel GreetingOptions {\n  timeOfDay: string;\n  style: string;\n}\n\nmodel UpdateNameOptions {\n  name: string;\n}\n\ninterface Greetin

In [6]:
prepare_res.typespec.typespec_definitions

'model UserInfo {\n  name: string;\n}\n\nmodel GreetingOptions {\n  timeOfDay: string;\n  style: string;\n}\n\nmodel UpdateNameOptions {\n  name: string;\n}\n\ninterface GreetingBot {\n  @scenario(\n  """\n  Scenario: Setting a user\'s name\n  When the user says "My name is John"\n  Then the system should extract "John" as the name\n  And store it for future greetings\n  Examples:\n      | Input                      | Expected Name |\n      | "Call me Sarah"            | "Sarah"       |\n      | "I am Robert Johnson"      | "Robert Johnson" |\n      | "You can refer to me as Alex" | "Alex"      |\n  """)\n  @llm_func("Store a user\'s name for personalized greetings")\n  setUserName(options: UpdateNameOptions): void;\n\n  @scenario(\n  """\n  Scenario: Greeting a user by name\n  When the user requests a greeting\n  Then the system should return a personalized greeting using their stored name\n  Examples:\n      | Stored Name | Time of Day | Expected Greeting               |\n      | "Jo

In [7]:
reasoning = prepare_res.typespec.reasoning
typespec_schema = prepare_res.typespec.typespec_definitions
fake_msg = f"<reasoning>{reasoning}</reasoning><typespec>{typespec_schema}</typespec>"

fake_msg

'<reasoning>The user requested a bot that greets them by name. This is a simple use case where the bot needs to:\n1. Store and retrieve the user\'s name\n2. Generate appropriate greetings based on the user\'s name\n3. Potentially support different greeting styles or times of day\n\nFor this bot, we need:\n- A model to represent the user\'s information (name)\n- A model for greeting options (time of day, style)\n- Functions to set the user\'s name and to generate greetings\n\nThe interface should support:\n- Setting the user\'s name\n- Getting a greeting with the user\'s name\n- Possibly updating greeting preferences</reasoning><typespec>model UserInfo {\n  name: string;\n}\n\nmodel GreetingOptions {\n  timeOfDay: string;\n  style: string;\n}\n\nmodel UpdateNameOptions {\n  name: string;\n}\n\ninterface GreetingBot {\n  @scenario(\n  """\n  Scenario: Setting a user\'s name\n  When the user says "My name is John"\n  Then the system should extract "John" as the name\n  And store it for fu

In [ ]:


final_res = builder.update_bot(
    typespec_schema=fake_msg,
    bot_id="1337_420",
    capabilities=[],
)

In [ ]:
final_res

In [ ]:
from core.interpolator import Interpolator

import os

# Create the botbuild directory if it doesn't exist
if not os.path.exists("../botbuild"):
    os.makedirs("../botbuild")

interpolator = Interpolator(".")

interpolator.bake(final_res, "../../bot_result")
